## お笑い（特に漫才）のテキストデータ分析

### 問い

1. 関西と関東で言葉の言い回しが違う？
    1. 関西は単純な言葉とノリで、関東はひねった言い回し？
2. 各芸人のネタを共起ネットワークに起こしてみると、芸人によって何か特徴が見られるか
3. ...

漫才のデータ分析をした記事もあったので参考にする。
感情分析はちょっと面白そう
- [データ分析で、M-1グランプリの"高得点パターン"を探してみた](https://note.com/natto_nebaneba/n/n11b4229f8b19)


### データの用意

ネタ台本の書き起こしをしているサイトからとってくる

- https://himapucchi.com/
- https://comedy.shivatax.com/category/manzaiwrite/
- https://manzaidaihon.com/
- https://bukiyo30.hateblo.jp/
- https://waraitext.com/
- https://hansode5.com/category/comedy-english/manzai/




# Code

### スクレイピング

https://himapucchi.com/
のスクレイピング

In [3]:
import os
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [29]:
# フォルダ
folder_name_raw = './data/raw'
print(os.path.exists(folder_name_raw))

folder_name_tidy = "./data/tidy"
print(os.path.exists(folder_name_tidy))

True
True


In [21]:
root_url = "https://himapucchi.com/"

In [22]:
# 17ページに分かれているのでlistを作成

crawl_list = []
crawl_list.append(root_url)

for i in range(16):
    crawl_list.append(f"{root_url}page/{i+2}")

crawl_list

['https://himapucchi.com/',
 'https://himapucchi.com/page/2',
 'https://himapucchi.com/page/3',
 'https://himapucchi.com/page/4',
 'https://himapucchi.com/page/5',
 'https://himapucchi.com/page/6',
 'https://himapucchi.com/page/7',
 'https://himapucchi.com/page/8',
 'https://himapucchi.com/page/9',
 'https://himapucchi.com/page/10',
 'https://himapucchi.com/page/11',
 'https://himapucchi.com/page/12',
 'https://himapucchi.com/page/13',
 'https://himapucchi.com/page/14',
 'https://himapucchi.com/page/15',
 'https://himapucchi.com/page/16',
 'https://himapucchi.com/page/17']

In [23]:
# <a>のlistを引数に、<a>内のURLとtitleを取得しdict listを返す
def extract_url_title_himapucchi(a_list):
    page_url_title = []
    for content in a_list:
        url = content.get('href')  # href属性を取得
        title = content.get("title")  # titleを取得
        page_url_title.append({"page_url": url, "title": title})
    return page_url_title


In [24]:
# User-Agent の設定
dummy_user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'


In [27]:
# htmlリクエスト & soup

page_url_title = []

for crawl_url in crawl_list:
    response = requests.get(crawl_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")

    page_list = soup.select("#main > #list > a")
    page_url_title.extend(extract_url_title_himapucchi(page_list))

    time.sleep(1) # 負荷軽減


In [28]:
page_df = pd.DataFrame(page_url_title)
page_df

,page_url,title
0,https://himapucchi.com/m2023-final-kurage/,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」
1,https://himapucchi.com/m2023-final-danbira/,【M-1グランプリ2023 1stラウンド】ダンビラムーチョ「カラオケボックス」
2,https://himapucchi.com/m2023-final-shinkujeshika/,【M-1グランプリ2023 1stラウンド】真空ジェシカ「B画館」
3,https://himapucchi.com/m2023-final-mayurika/,【M-1グランプリ2023 1stラウンド】マユリカ「倦怠期」
4,https://himapucchi.com/m2023-final-kabeposutar/,【M-1グランプリ2023 1stラウンド】カベポスター「おまじない」
...,...,...
163,https://himapucchi.com/m2019first-milkboy/,【M-1グランプリ2019 1stラウンド】<br>ミルクボーイ「コーンフレーク」
164,https://himapucchi.com/m2019first-kamaitachi/,【M-1グランプリ2019 1stラウンド】<br>かまいたち「UFJ」
165,https://himapucchi.com/m2019final-milkboy/,【M-1グランプリ2019 最終決戦】<br>ミルクボーイ「最中」
166,https://himapucchi.com/m2019final-kamaitachi/,【M-1グランプリ2019 最終決戦】<br>かまいたち「トトロ」


In [31]:
# いったんcsvに保存
page_df.to_csv(os.path.join(folder_name_raw, "page_url_title.csv"), index=False)

In [52]:
# <div>listからscriptの抽出
def extract_script_himapucchi(script_div_list, title):
    script_dict_list = []
    name_pos_r = "" # 右側の芸人の名前
    name_pos_l = "" # 左側の芸人の名前
    for i, script_div in enumerate(script_div_list):
        clas = script_div.get("class")
        text = script_div.select_one("p")
        if text:
            text = text.get_text(strip=True)
        name = script_div.select_one("div.speech-name")
        if name:
            name = name.get_text(strip=True)
            if "sbp-r" in clas:
                name_pos_r = name
                print("r: " + name_pos_r)
            elif "sbp-l" in clas:
                name_pos_l = name
                print("l: " + name_pos_l)
            else:
                raise "invalid class (name)"
        
        if "sbp-r" in clas: # 右側の芸人のセリフ
            script_dict_list.append({"title": title, "pos": "r", "name": name_pos_r, "turn": i, "speech": text})
        elif "sbp-l" in clas: # 左側の芸人のセリフ
            script_dict_list.append({"title": title, "pos": "l", "name": name_pos_l, "turn": i, "speech": text})
        else:
            raise "invalid class"
    
    return script_dict_list
            

# ページからtitleとscriptを抽出
def extract_title_script_himapucchi(page_url):
    response = requests.get(page_url, headers={"User-Agent": dummy_user_agent}, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.select_one(".entry-title").get_text(strip=True)
    print(title)

    script_div_list = soup.select(".entry-content > div.speech-wrap")
    script_dict_list = extract_script_himapucchi(script_div_list, title)

    return script_dict_list

    

In [53]:
# scriptのスクレイピング

script_df_all = pd.DataFrame()
for page_url in page_df["page_url"]:
    script_df = pd.DataFrame(extract_title_script_himapucchi(page_url))
    script_df_all = pd.concat([script_df_all, script_df], ignore_index=True)
    time.sleep(1) # 負荷軽減

script_df_all

【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」
r: 渡辺
l: 杉
【M-1グランプリ2023 1stラウンド】ダンビラムーチョ「カラオケボックス」
【M-1グランプリ2023 1stラウンド】真空ジェシカ「B画館」
r: 川北
l: ガク
【M-1グランプリ2023 1stラウンド】マユリカ「倦怠期」
r: 中谷
l: 阪本
【M-1グランプリ2023 1stラウンド】カベポスター「おまじない」
r: 浜田
l: 永見
【M-1グランプリ2023 1stラウンド】シシガシラ「規制」
r: 脇田
l: 浜中
【M-1グランプリ2023 最終決戦】さや香「見せ算」
r: 石井
l: 新山
【M-1グランプリ2023 最終決戦】ヤーレンズ「ラーメン屋」
r: 出井
l: 楢原
【M-1グランプリ2023 1stラウンド】ヤーレンズ「引越しの挨拶」
r: 出井
l: 楢原
【M-1グランプリ2023 1stラウンド】さや香「ホームステイ」
l: 新山
r: 石井
【M-1グランプリ2023 最終決戦】令和ロマン「ドラマ」
r: 松井
l: 髙比良
【M-1グランプリ2023 1stラウンド】令和ロマン「少女漫画」
r: 松井
l: 髙比良
【M-1グランプリ2023 3回戦】ヤーレンズ「ケーキ屋」
r: 出井
l: 楢原
【M-1グランプリ2023 3回戦】マユリカ「井戸端会議」
r: 中谷
l: 阪本
【M-1グランプリ2023 3回戦】令和ロマン「100m走」
r: 松井
l: 髙比良
【M-1グランプリ2023 3回戦】ナイチンゲールダンス「居酒屋」
r: ヤス
l: 中野
【M-1グランプリ2023 3回戦】くらげ「不幸」
l: 杉
r: 渡辺
【M-1グランプリ2023 3回戦】ダンビラムーチョ「昔話」
l: 大原
r: 原田
【M-1グランプリ2023 3回戦】エバース「略奪」
r: 町田
l: 佐々木
【M-1グランプリ2023 3回戦】シシガシラ「同窓会」
r: 脇田
l: 浜中
【M-1グランプリ2023 3回戦】ぎょうぶ「家族」
【M-1グランプリ2023 3回戦】ななまがり「マッチングアプリ」
【M-1グランプリ2023 3回戦】ママタルト「ラブコメ」
【M-1グランプリ2023 3回戦】さや香「大学」
l: 新山
r: 石井
【M-

,title,pos,name,turn,speech
0,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,0,ども！くらげです。お願いします
1,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,1,お願いします
2,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,2,すみません、ありがとうございます！頑張っていきましょうねー
3,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,l,杉,3,あのー、俺さ、この前初めてさ、31アイスに行ったのよ
4,【M-1グランプリ2023 1stラウンド】くらげ「物忘れ」,r,渡辺,4,あ、31。でも31ってさ、店内カラフルだし女子高生とか多いからなんか、おじさんの俺達は入りづらいな
...,...,...,...,...,...
24680,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,89,ちょっとちょっと、ちょっと待ってくれ相方！
24681,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,90,え？
24682,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,r,松陰寺,91,ずっと何やってんだよ！
24683,【M-1グランプリ2019 最終決戦】ぺこぱ「お年寄り」,l,シュウペイ,92,いい加減にしろ！


In [54]:
script_df_all.to_csv(os.path.join(folder_name_raw, "script_all.csv"), index=False)